In [8]:
import numpy as np
import scipy
import numpy 
import matplotlib.pyplot as plt
from scipy import linalg, special, stats
from numpy import genfromtxt
import ML_support as ml

In [11]:
Data, labels = ml.loadFile('../Train.txt')
n, m = sum(labels == 1), sum(labels == 0)
print('Prima ', n/m)
numpy.random.seed(0)
indexes = numpy.random.permutation(1200)
Data_s = Data[:, indexes]
label_s = labels[indexes]
ns, ms = sum(label_s == 1), sum(label_s == 0)
print('Dopo :', ns/ms)

Prima  0.10125801677355697
Dopo : 0.10497237569060773


In [12]:
(DTR, LTR), (DTE, LTE) = ml.split_db_2to1(Data_s, label_s, seed=42)
n1, n0 = sum(LTR == 1), sum(LTR == 0)
print('Dopo dopo', n1/n0)


Dopo dopo 0.11265646731571627
